# This script contains the following:
- Import libraries
- Load datasets
- Checking and wrangling dataframes
- Find the mean orders for each department
- Grouping data per max order to prepare flag creation
- Creating loyalty flag 
- Compare customer's spending habits depending of their loyalty
- Create flags: Types of spenders 
- Create flags: Frequent customer
- Export

# Import libraries & datasets

In [34]:
import pandas as pd
import numpy as np
import os

# Load dataframes

In [35]:
# Define full absolute paths
path = r'/Users/macbook/Dropbox/Mac/Documents/Pro/Data_Analyst/Course_Career_foundry/A4_Python/2023.08_Instacart_basket_analysis'
department_file_path = os.path.join(path, 'prepared data', 'departments_wrangled.csv')
orders_file_path = os.path.join(path, 'prepared data', 'orders_products_merged_v2')


# Import the dataframe
df_dept = pd.read_csv(department_file_path, index_col=False)
df_ords = pd.read_pickle(orders_file_path)  # Use pd.read_pickle() for pickle files

# Checking and wrangling dataframes

In [36]:
# Checking import
df_ords.head()

,order_id,user_id,number_order_client,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,both,Soda,77,7,9.0,both,mid-range product,regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,both,Soda,77,7,9.0,both,mid-range product,regularly busy,Least busy days,Average orders
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,both,Soda,77,7,9.0,both,mid-range product,regularly busy,Least busy days,Most orders
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,both,Soda,77,7,9.0,both,mid-range product,least busy,Least busy days,Average orders
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,both,Soda,77,7,9.0,both,mid-range product,least busy,Least busy days,Most orders


In [37]:
# Drop the '_merge' and 'merge_indicator' columns from the dataframe 'df_ords'
df_ords = df_ords.drop(['_merge', 'merge_indicator'], axis=1)

In [38]:
# Checking drop columns
df_ords.head()

,order_id,user_id,number_order_client,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,77,7,9.0,mid-range product,regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,77,7,9.0,mid-range product,regularly busy,Least busy days,Average orders
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,77,7,9.0,mid-range product,regularly busy,Least busy days,Most orders
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,77,7,9.0,mid-range product,least busy,Least busy days,Average orders
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,77,7,9.0,mid-range product,least busy,Least busy days,Most orders


In [39]:
df_ords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   number_order_client     int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              float64
 7   add_to_cart_order       float64
 8   reordered               float64
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  busiest_day             object 
 15  busiest_days            object 
 16  busiest_period_of_day   object 
dtypes: float64(5), int64(7), object(5)
memory usage: 4.3+ GB


In [40]:
# Checking import department dataframe
df_dept

,Unnamed: 0,department_id,department
0,1,1,frozen
1,2,2,other
2,3,3,bakery
3,4,4,produce
4,5,5,alcohol
5,6,6,international
6,7,7,beverages
7,8,8,pets
8,9,9,dry goods pasta
9,10,10,bulk


In [41]:
# drop 'Unnamed: 0' from dept dataframe
df_dept = df_dept.drop('Unnamed: 0', axis=1)
df_dept.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


# Find the mean orders for each department

In [42]:
# Group by product name
df_ords.groupby('product_name')

In [43]:
# group by "department_id" and calculate the mean for each "order_number" column
round(df_ords.groupby('department_id').agg({'number_order_client': ['mean', 'min','max']}))

number_order_client        
                             mean min max
department_id                            
1                            15.0   1  99
2                            17.0   1  99
3                            17.0   1  99
4                            18.0   1  99
5                            15.0   1  99
6                            16.0   1  99
7                            17.0   1  99
8                            15.0   1  99
9                            16.0   1  99
10                           20.0   1  99
11                           16.0   1  99
12                           16.0   1  99
13                           17.0   1  99
14                           17.0   1  99
15                           16.0   1  99
16                           18.0   1  99
17                           16.0   1  99
18                           19.0   1  99
19                           17.0   1  99
20                           16.0   1  99
21                           23.0   1  99

the range is wider with the entire data frame with a mean between 15 to 23 

# Grouping data per max order to prepare flag creation

In [44]:
# Group data by "user_id" column
df_ords['max_order'] = df_ords.groupby(['user_id'])['number_order_client'].transform(np.max)

In [45]:
# Setting the maximum number of displayed rows for pandas DataFrame to unlimited
pd.options.display.max_rows = None

In [46]:
df_ords.columns

Index(['order_id', 'user_id', 'number_order_client', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order'],
      dtype='object')

In [47]:
# checking results
df_ords[['user_id','max_order', 'department_id']].head(50)

,user_id,max_order,department_id
0,1,10,7
1,1,10,7
2,1,10,7
3,1,10,7
4,1,10,7
5,1,10,7
6,1,10,7
7,1,10,7
8,1,10,7
9,1,10,7


# Creating Loyalty flags

In [48]:
# Defining loyal customer flag
df_ords.loc[df_ords['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [49]:
# Defining regular customer flag
df_ords.loc[(df_ords['max_order'] <= 40) & (df_ords['max_order'] > 10), 'loyalty_flag'] = 'regular customer'

In [50]:
# Defining new customer flag
df_ords.loc[df_ords ['max_order']<= 10, 'loyalty_flag'] = 'new customer'

In [51]:
# Printing flags
df_ords['loyalty_flag'].value_counts(dropna = False)

regular customer    15876776
Loyal customer      10284093
new customer         6243990
Name: loyalty_flag, dtype: int64

In [52]:
# Checking loyalty flags created
df_ords[['user_id', 'loyalty_flag', 'number_order_client']].head(60)

,user_id,loyalty_flag,number_order_client
0,1,new customer,1
1,1,new customer,2
2,1,new customer,3
3,1,new customer,4
4,1,new customer,5
5,1,new customer,6
6,1,new customer,7
7,1,new customer,8
8,1,new customer,9
9,1,new customer,10


# Compare customer's spending habits depending of their loyalty

In [53]:
# Group by "prices" 
df_ords.groupby('prices')

In [54]:
round(df_ords.groupby('loyalty_flag').agg({'prices': ['mean','median', 'min','max']}))

prices                     
                   mean median  min      max
loyalty_flag                                
Loyal customer     10.0    7.0  1.0  99999.0
new customer       13.0    7.0  1.0  99999.0
regular customer   12.0    7.0  1.0  99999.0

NOTE: it looks like loyal customers are buying more products but cheaper. We also have an outliers detected before. it affects the mean prices. If we look at the median, it is similar.

# Create flags: Types of spenders 

In [55]:
# Grouping by prices 
df_ords['mean_price'] = df_ords.groupby(['user_id'])['prices'].transform(np.mean)

In [56]:
# Create flags low spender customer
df_ords.loc[df_ords['mean_price'] < 10, 'spending_flag'] = 'Low spender'

In [57]:
# Create flags high spender customer
df_ords.loc[df_ords['mean_price'] >= 10, 'spending_flag'] = 'High spender'

In [58]:
# Check frequencies of spending flags
df_ords['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

# Create flags: Frequent customer

In [59]:
# grouping per days since prior
df_ords['median_days'] = df_ords.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [60]:
# creating flag for non frequent customer 
df_ords.loc[df_ords['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [61]:
# creating flag for regular customer
df_ords.loc[(df_ords['median_days'] > 10) & (df_ords['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

In [62]:
# creating flag for frequent customer 
df_ords.loc[df_ords['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [63]:
# Check frequencies
df_ords['frequency_flag'].value_counts(dropna=False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

# Export

In [65]:
# Export data to pkl
df_ords.to_pickle(os.path.join(path, 'prepared data','orders_products_aggregated.pkl'))